# 📊 Анализ обучения Seq2Seq модели

Визуализация процесса обучения модели для чат-бота МУИВ

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path

# Стиль графиков
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Загрузка данных обучения

In [ ]:
# Путь к модели
model_path = '../ml/models/chatbot_model.pt'

# Загружаем checkpoint
checkpoint = torch.load(model_path, map_location='cpu')

# Извлекаем историю обучения
train_losses = checkpoint.get('train_losses', [])
val_losses = checkpoint.get('val_losses', [])

print(f"✅ Загружены данные обучения")
print(f"   Эпох обучения: {len(train_losses)}")
print(f"   Финальный Train Loss: {train_losses[-1]:.4f}")
if val_losses:
    print(f"   Финальный Val Loss: {val_losses[-1]:.4f}")

## 2. График Loss

In [ ]:
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 6))
plt.plot(epochs, train_losses, 'b-', label='Train Loss', linewidth=2)
if val_losses:
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)

plt.xlabel('Эпоха', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Динамика обучения модели', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('training_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 График сохранён: training_loss.png")

## 3. Статистика обучения

In [ ]:
# Создаём DataFrame
df = pd.DataFrame({
    'Эпоха': epochs,
    'Train Loss': train_losses,
    'Val Loss': val_losses if val_losses else [None] * len(train_losses)
})

print("📊 Статистика обучения:")
print(df.describe())

# Лучшая эпоха
if val_losses:
    best_epoch = np.argmin(val_losses) + 1
    best_val_loss = min(val_losses)
    print(f"\n✅ Лучшая эпоха: {best_epoch}")
    print(f"   Val Loss: {best_val_loss:.4f}")

## 4. Информация о модели

In [ ]:
print("🏗️ Архитектура модели:")
print(f"   Vocab size: {checkpoint['vocab_size']:,}")
print(f"   Embedding dim: {checkpoint['embedding_dim']}")
print(f"   Hidden size: {checkpoint['hidden_size']}")
print(f"   Num layers: {checkpoint['num_layers']}")
print(f"   Dropout: {checkpoint['dropout']}")

# Подсчёт параметров
total_params = sum(p.numel() for p in checkpoint['model_state_dict'].values())
print(f"\n📊 Всего параметров: {total_params:,}")
print(f"   Размер модели: {total_params * 4 / 1024 / 1024:.2f} MB (float32)")

## 5. График улучшения

In [ ]:
if val_losses:
    # Вычисляем улучшение относительно первой эпохи
    initial_val_loss = val_losses[0]
    improvement = [(initial_val_loss - loss) / initial_val_loss * 100 for loss in val_losses]
    
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, improvement, 'g-', linewidth=2, marker='o')
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
    
    plt.xlabel('Эпоха', fontsize=14)
    plt.ylabel('Улучшение Val Loss (%)', fontsize=14)
    plt.title('Прогресс обучения относительно первой эпохи', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('training_improvement.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 График сохранён: training_improvement.png")
    print(f"\n📈 Итоговое улучшение: {improvement[-1]:.2f}%")

## 6. Экспорт результатов

In [ ]:
# Сохраняем результаты в CSV
df.to_csv('training_results.csv', index=False, encoding='utf-8')
print("💾 Результаты сохранены: training_results.csv")

# Создаём отчёт
report = f"""
ОТЧЁТ ОБ ОБУЧЕНИИ МОДЕЛИ ЧАТ-БОТА
{'=' * 50}

Архитектура:
  - Модель: Seq2Seq с Attention
  - Encoder: {checkpoint['num_layers']}-слойный LSTM
  - Decoder: {checkpoint['num_layers']}-слойный LSTM + Bahdanau Attention
  - Размер словаря: {checkpoint['vocab_size']:,}
  - Параметров: {total_params:,}

Обучение:
  - Эпох: {len(train_losses)}
  - Финальный Train Loss: {train_losses[-1]:.4f}
""" + (f"  - Финальный Val Loss: {val_losses[-1]:.4f}\n" if val_losses else "") + (
f"  - Лучший Val Loss: {min(val_losses):.4f} (эпоха {np.argmin(val_losses) + 1})\n" if val_losses else ""
) + f"""
Результат:
  ✅ Модель обучена успешно
  ✅ Готова к использованию в боте
"""

with open('training_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("💾 Отчёт сохранён: training_report.txt")
print(report)